In [26]:
import pandas as pd
from pandas import TimeSeries
import numpy as np
import math
from bokeh.charts import TimeSeries, output_file, show
import statsmodels.tsa.stattools as stat

from scipy import stats
import matplotlib.pyplot as plt
from math import *
from datetime import datetime
from datetime import timedelta

%matplotlib inline

In [10]:
sp500=pd.read_csv('sp_500_daily_prices.csv',usecols=['date','COMNAM','PRC',\
                    'RET','ASK','BID'],engine='c')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
start=20130101
end=20150101

In [12]:
sp_recent=sp500[sp500['date']>=start]
sp_recent=sp_recent[sp_recent['date']<=end]

In [15]:
def data_process(start,end):
    sp500=pd.read_csv('sp_500_daily_prices.csv',usecols=['date','COMNAM','PRC',\
                    'RET','ASK','BID'],engine='c')
    sp_recent=sp500[sp500['date']>=start]
    sp_recent=sp_recent[sp_recent['date']<=end]
    
    date_list=sorted(list(sp_recent['date']))
    company_recent=list(sp_recent[sp_recent['date']==date_list[0]]['COMNAM'])
    sp_orig=sp_recent[sp_recent['COMNAM']==company_recent[0]][['date','RET']]
    sp_orig=pd.DataFrame(list(sp_orig['RET']),columns=[company_recent[0]],index=list(sp_orig['date']))
    sp_orig=sp_orig[~sp_orig.index.duplicated(keep='first')]
    for i in range(1,len(company_recent)):
        company=company_recent[i]
        current_data=sp_recent[sp_recent['COMNAM']==company][['date','RET']]
        ret=pd.DataFrame(list(current_data['RET']),columns=[company],index=list(current_data['date']))
        ret =ret[~ret.index.duplicated(keep='first')]
        sp_orig=pd.concat([sp_orig,ret],join='outer',axis=1)
    sp_ret=sp_orig.loc[date_list[0]:date_list[int(0.9*len(date_list))],:]
    sp_ret=sp_ret.dropna(axis=1)
    
    
    sp_price=sp_recent[sp_recent['COMNAM']==company_recent[-1]][['date','PRC']]
    sp_price=pd.DataFrame(list(sp_price['PRC']),columns=[company_recent[-1]],index=list(sp_price['date']))
    sp_price=sp_price[~sp_price.index.duplicated(keep='first')]
    for company in company_recent:
        current_data=sp_recent[sp_recent['COMNAM']==company][['date','PRC']]
        price=pd.DataFrame(list(current_data['PRC']),columns=[company],index=list(current_data['date']))
        price =price[~price.index.duplicated(keep='first')]
        sp_price=pd.concat([sp_price,price],join='outer',axis=1)
    sp_price=sp_price.loc[date_list[0]:date_list[int(0.9*len(date_list))],:]
    
    return sp500,sp_ret,sp_price,date_list


In [16]:
def filter_pair(pairs):
    selected_pair=[]
    stock_list=[]
    for pair in pairs:
        if pair[0] not in stock_list and pair[1] not in stock_list:
            selected_pair+=[pair]
            stock_list+=pair
    return selected_pair

In [17]:
def find_pair(start,end):
    sp500,sp_ret,sp_price,date_list=data_process(start,end)
    error_return=['E','D','C','B','A','']
    sp_ret=sp_ret.applymap(lambda x:np.float64(0.0) if x in error_return else np.float64(x))
    sp_pca=np.array(sp_ret)
    cov_company=np.cov(sp_pca)
    cov_company[np.isnan(cov_company)] = 0.0
    eigen_val,eigen_vector=np.linalg.eig(cov_company)
    eigen_vector_max=list(zip(eigen_vector[np.argmax(eigen_val)],sp_ret.columns))
    eigen_vector_max.sort(key=lambda x:x[0],reverse=True)
    eigen_vector_max=list(zip(*eigen_vector_max))
    data=sp_price[list(eigen_vector_max[1][0:30])]
    score_matrix, pvalue_matrix, pairs=find_cointegrated_pairs(data)
    selected_pair=filter_pair(pairs)
    data_inuse=sp_recent[sp_recent['date']>=date_list[int(0.9*len(date_list))]]
    data_inuse=data_inuse[data_inuse['date']<=end]
    return data_inuse,selected_pair


In [18]:
def find_cointegrated_pairs(data):
    n = len(data.columns)
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = list(data.columns)
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = np.array(data[keys[i]])
            S2 = np.array(data[keys[j]])
            result = stat.coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs

In [19]:
data,pairs=find_pair(start,end)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:1546: RuntimeWarning: invalid value encountered in greater
  return sum(S > tol)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1731: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


In [20]:
pairs

[('GENESCO INC', 'HARRIS CORP'),
 ('COMPUWARE CORP', 'BARD C R INC'),
 ('CINCINNATI FINANCIAL CORP', 'TERADYNE INC'),
 ('PEPSICO INC', 'GENERAL GROWTH PPTYS INC NEW'),
 ('CROWN HOLDINGS INC', 'A G L RESOURCES INC'),
 ('BASSETT FURNITURE INDUSTRIES INC', 'GENERAL ELECTRIC CO')]

In [18]:
data

,date,COMNAM,PRC,RET,BID,ASK
12956,20141020,ORACLE CORP,37.80,-0.001848,37.79,37.80
12957,20141021,ORACLE CORP,38.35,0.01455,38.34,38.35
12958,20141022,ORACLE CORP,37.64,-0.018514,37.64,37.65
12959,20141023,ORACLE CORP,38.23,0.015675,38.22,38.23
12960,20141024,ORACLE CORP,38.73,0.013079,38.72,38.73
12961,20141027,ORACLE CORP,38.43,-0.007746,38.42,38.43
12962,20141028,ORACLE CORP,38.65,0.005725,38.62,38.63
12963,20141029,ORACLE CORP,38.58,-0.001811,38.58,38.59
12964,20141030,ORACLE CORP,38.50,-0.002074,38.50,38.51
12965,20141031,ORACLE CORP,39.05,0.014286,39.01,39.02


In [21]:
signal_exists = ['Y','N']         # exist, not exist
signal_opens  = ['O', 'C']        # open, close
signal_moves  = ['B', 'S']        # buy, sell

In [22]:
def generate_signal(priceA, priceB):
    '''Find the trading signal in one window, default as 60 days'''
    
    # test the existence of signal
    tfA = len(np.unique(np.isnan(priceA)))
    tfB = len(np.unique(np.isnan(priceB)))
            
    if  tfA == 2 or tfB == 2:
        signal_exist = signal_exists[0]
    else:
        signal_exist = signal_exists[1]

        # calculate return for two stocks
        Areturn = np.diff(priceA)/priceA[:(len(priceA)-1)]
        Breturn = np.diff(priceB)/priceB[:(len(priceB)-1)]

        # regression on return
        beta, beta0, r_value, p_value, std_err = stats.linregress(Areturn, Breturn)

        # get the residual epsilon_t
        e_t = np.array(Breturn - beta0 - beta*Areturn)

        # auxiliary process X_t
        Xt = []
        for i in range(len(Areturn)):
            Xt.append(np.sum(e_t[:i+1]))

        # regression on X_t
        length = len(Xt)
        Xt_vec = np.array(Xt)
        b, a, r_value_x, p_value_x, std_err_x = stats.linregress(Xt_vec[1:],Xt_vec[:length-1])

        # get the residual zeta_t
        z_t = Xt_vec[:length-1] - a - b*Xt_vec[1:]
        var_z = np.var(z_t)

        # calculate the s-score
        s_score = -a*sqrt(1-b**2)/((1-b)*sqrt(var_z)) + int(a/(1-b))*sqrt((1-b**2)/var_z)

        # trading signal
        if s_score < -1.25:
            signal_open = signal_opens[0]
            signal_move = signal_moves[0]
        elif s_score > 1.25:
            signal_open = signal_opens[0]
            signal_move = signal_moves[1]
        elif s_score < 0.75:
            signal_open = signal_opens[1]
            signal_move = signal_moves[0]
        elif s_score > -0.5:
            signal_open = signal_opens[1]
            signal_move = signal_moves[1]
        else:
            print('warning!')

        return beta,signal_exist, signal_open, signal_move

In [29]:
position_dict = {}  # static variable: 'StockA': position
cash = 50000        # static variable


def build_position(position_dict, cash, pairs, data):
    '''build position for each window
    
    Default parameters
    ------------------
    shares per trade:1000
    transaction cost fee: 0.0005
    '''
    
    transaction_cost = 0.0005
    PNL = []
    
    for i in range(60):
        cash_old = cash
        for pair in pairs:
            stockA         = data[data['COMNAM'] == pair[0]]
            stockB         = [data['COMNAM'] == pair[1]]
            priceA         = pd.DataFrame.as_matrix(stockA['PRC'])
            priceB         = pd.DataFrame.as_matrix(stockB['PRC'])
            tickerA        = pair[0]
            tickerB        = pair[1]
            BidA           = pd.DataFrame.as_matrix(stockA['BID'])
            AskA           = pd.DataFrame.as_matrix(stockA['ASK'])
            BidB           = pd.DataFrame.as_matrix(stockB['BID'])
            AskB           = pd.DataFrame.as_matrix(stockB['ASK'])
            current_priceA = priceA[i:i+window]
            current_priceB = priceB[i:i+window]
            result         = generate_signal(current_priceA, current_priceB)
            beta           = result[0]
            open_or_not    = result[2]
            buy_or_sell    = result[3]
            
            # for A
            if (tickerA in position_dict.keys()) & (position_dict[tickerA] != 0):
                if open_or_not == 'C':
                    if buy_or_sell == 'B':
                        cash += (BidA[i+60]-transaction_cost)*position_dict[tickerA]
                        position_dict[tickerA] = 0
                    else:
                        cash -= (AskA[i+60]+transaction_cost)*position_dict[tickerA]
                        position_dict[tickerA] = 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerA] -= int((cash/len(pairs))/AskB[i+60]*beta)
                        cash += (asset_value/len(pairs))/AskB[i+60]*beta*(BidA[i+60]-transaction_cost)
                    else:
                        position_dict[tickerA] += int((cash/len(pairs))/BidB[i+60]*beta)
                        cash -= (asset_value/len(pairs))/BidB[i+60]*beta*(AskA[i+60]+transaction_cost)
            else:
                if open_or_not == 'C':
                    position_dict[tickerA] = 0
                    cash += 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerA] = -int((cash/len(pairs))/AskB[i+60]*beta)
                        cash += abs(position_dict[tickerA])*(BidA[i+60]-transaction_cost)
                    else:
                        position_dict[tickerA] = int((cash/len(pairs))/BidB[i+60]*beta)
                        cash -= abs(position_dict[tickerA])*(AskA[i+60]+transaction_cost)
            
            # for B
            if (tickerB in position_dict.keys()) & (position_dict[tickerB] !=0):
                if open_or_not == 'C':
                    if buy_or_sell == 'B':
                        cash -= (AskB[i+60]+transaction_cost)*position_dict[tickerB]
                        position_dict[tickerB] = 0
                    else:
                        cash += (BidB[i+60]-transaction_cost)*position_dict[tickerB]
                        position_dict[tickerB] = 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerB] += int((cash/len(pairs))/AskB[i+60])
                        cash -= (asset_value/len(pairs))/AskB[i+60]*(AskB[i+60]+transaction_cost)
                    else:
                        position_dict[tickerB] -= int((cash/len(pairs))/BidB[i+60])
                        cash += (asset_value/len(pairs))/BidB[i+60]*(BidB[i+60]-transaction_cost)
            else:
                if open_or_not == 'C':
                    position_dict[tickerB] = 0
                    cash += 0
                else:
                    if buy_or_sell == 'B':
                        position_dict[tickerB] = int((cash/len(pairs))/AskB[i+60])
                        cash -= abs(position_dict[tickerB])*(AskB[i+60]+transaction_cost)
                    else:
                        position_dict[tickerB] = int((cash/len(pairs))/BidB[i+60])
                        cash += abs(position_dict[tickerB])*(BidB[i+60]-transaction_cost)
        PNL.append(cash - cash_old)    
    return position_dict, cash, PNL

In [24]:
def add_date(start_date, add_days):
    year = int(start_date/10000)
    month = int(start_date/100)- year*100
    day = start - year*10000 - month*100
    
    date1 = datetime(year, month, day)
    date2 = date1 + timedelta(days=add_days)
    
    new_date  = date2.year * 10000 + date2.month * 100 + date2.day
    return new_date
    

In [ ]:
def backtest(start_date, timeline):
    win_len = 60
    pair_start_date = add_date(start_date, -win_len*9)
    pair_end_date = add_date(start_date, win_len - 1)
    
    
    PNL_list = []
    
    for i in range(timeline):
        data, pairs = find_pair(pair_start_date, pair_end_date)
        PNL = build_position(position_dict, cash, pairs, data)[2]
        PNL_list = PNL_list + PNL
        pair_start_date = add_date(pair_start_date, win_len)
        pair_end_date = add_date(pair_end_date, win_len)
    
    return PNL_list
      

In [47]:
backtest(sp500)

6553
priceA:
priceB:
a:
b:


ValueError: cannot convert float NaN to integer